## When nothing works, Boosting does. Nowadays many people use either XGBoost or LightGBM or CatBoost to win competitions at Kaggle or Hackathons. AdaBoost is the first stepping stone in the world of Boosting.
## So here in this notebook I will take about Adaboost and Gradient Boost.


### 1- Importing data and libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [ ]:
df_solar= pd.read_csv('../input/SolarEnergy/SolarPrediction.csv')
df_solar

### Quick visualization:

In [ ]:
sns.distplot(df_solar['UNIXTime'])

In [ ]:
df_solar=df_solar.drop(['Data','Time','TimeSunRise','TimeSunSet'],axis=1)
df_solar

In [ ]:
figure= plt.figure(figsize=(10,10))
sns.heatmap(df_solar.corr(),annot=True)

In [ ]:
figure= plt.figure(figsize=(20,10))
sns.pairplot(df_solar)

### Data Frame to cotain regression Evaluation:


In [ ]:
X= df_solar.drop('Radiation',axis=1)
y=df_solar['Radiation']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.30,random_state=101)

In [ ]:
from sklearn import metrics
from sklearn.model_selection import cross_val_score

results_df = pd.DataFrame()
columns = ["Model", "Cross Val Score", "MAE", "MSE", "RMSE", "R2"]

def evaluate(true, predicted):
    mae = metrics.mean_absolute_error(true, predicted)
    mse = metrics.mean_squared_error(true, predicted)
    rmse = np.sqrt(metrics.mean_squared_error(true, predicted))
    r2_square = metrics.r2_score(true, predicted)
    return mae, mse, rmse, r2_square

def append_results(model_name, model, results_df, y_test, pred):
    results_append_df = pd.DataFrame(data=[[model_name, *evaluate(y_test, pred) , cross_val_score(model, X, y, cv=10).mean()]], columns=columns)
    results_df = results_df.append(results_append_df, ignore_index = True)
    return results_df

## 1- Regression AdaBoost :
#### AdaBoost is one of the first boosting algorithms to be adapted in solving practices. Adaboost helps you combine multiple “weak classifiers” into a single “strong classifier”
####  → The weak learners in AdaBoost are decision trees with a single split, called decision stumps.
#### → AdaBoost works by putting more weight on difficult to classify instances and less on those already handled well.
#### → AdaBoost algorithms can be used for both classification and regression problem.

In [ ]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import RandomForestRegressor
ada_reg= AdaBoostRegressor(RandomForestRegressor())
ada_reg.fit(X_train,y_train)

In [ ]:
pred = ada_reg.predict(X_test)

In [ ]:
results_df= append_results("AdaBoost Regression",AdaBoostRegressor(RandomForestRegressor()),results_df,y_test,pred)
results_df

### To see the normal distribution between y_test and prediction
#### If it a correct model it will normally distributed with y_test

In [ ]:
sns.distplot((y_test,pred))

## 2- Gradient Boosting Regression
#### Gradient Boosting trains many models in a gradual, additive and sequential manner. The major difference between AdaBoost and Gradient Boosting Algorithm is how the two algorithms identify the shortcomings of weak learners (eg. decision trees).
#### While the AdaBoost model identifies the shortcomings by using high weight data points, gradient boosting performs the same by using gradients in the loss function (y=ax+b+e , e needs a special mention as it is the error term). The loss function is a measure indicating how good are model’s coefficients are at fitting the underlying data.

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
gr_reg= GradientBoostingRegressor()
gr_reg.fit(X_train,y_train)
pred = gr_reg.predict(X_test)

### To see the normal distribution between y_test and prediction
#### If it a correct model it will normally distributed with y_test

In [ ]:
sns.distplot((y_test,pred))

In [ ]:
results_df= append_results("Gradient Regression",GradientBoostingRegressor(),results_df,y_test,pred)
results_df

### So that in our case Gradiet Boosting is better than AdaBoost!

source1 : https://towardsdatascience.com/understanding-gradient-boosting-machines-9be756fe76ab

source2: https://towardsdatascience.com/understanding-adaboost-2f94f22d5bfe